In [45]:
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import os
import numpy as np 
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# # 设置plotly默认主题
# pio.templates.default = 'plotly_white'


In [46]:
def read_tensorboard_data(tensorboard_path, val_name):
    """读取tensorboard数据，
    tensorboard_path是tensorboard数据地址val_name是需要读取的变量名称"""
    ea = event_accumulator.EventAccumulator(tensorboard_path)
    ea.Reload()
    print(ea.scalars.Keys())
    val = ea.scalars.Items(val_name)
    return val
    
def get_event_data(tensorboard_path,key):
    # print(tensorboard_path)
    ea = event_accumulator.EventAccumulator(tensorboard_path)
    ea.Reload()
    # print(ea.scalars.Keys())
    # print(key)
    value = [i.value for i in  ea.scalars.Items(key)]
    return value

def get_file_list(file_path):
    dir_list = os.listdir(file_path)
    if not dir_list:
        return
    else:
        # 注意，这里使用lambda表达式，将文件按照最后修改时间顺序升序排列
        # os.path.getmtime() 函数是获取文件最后修改时间
        # os.path.getctime() 函数是获取文件最后创建时间
        dir_list = sorted(dir_list,key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
        # print(dir_list)
        return dir_list

def get_event_data_mean(tensorboard_path_list,key):
    len_ea = len(tensorboard_path_list)
    eas = [event_accumulator.EventAccumulator(i).Reload() for i in tensorboard_path_list]
    values = []
    for ea in eas:
        value = [i.value for i in ea.scalars.Items(key)]
        values.append(value)
    
    values = np.array(values)
    values = list(np.mean(values,axis=0))
    return values

def get_group_data(dataset='mnist',distribution='iid',attack_type='label',define_type='fedavg',key='Server_Acc',attack_round=150,mean_value=False):
    '''
        attack_type = ['gaussian','label','pixel','replace']
        define_type = ['fedavg','fedavg-krum','fedsign','fedsign-krum','fedsign-fadc-nmi','fedsign-fadc-ed']
    '''

    if dataset == 'cifar10':
        model_n = 'alexnet'
    else:
        model_n = 'lanet'

    if define_type == 'majority-vote' or define_type == 'majority-vote-ad' :
        r_round = 4000
        attack_round=3500
    elif define_type == 'dprsa-f' or define_type == 'dprsa-g':
        r_round =1000
        attack_round = 800
    elif define_type in ['fedsign-mipc-mask','fedavg','fedavg-krum'] and dataset == 'cifar10':
        r_round=300
        attack_round = 250
    else:
        r_round = 200

    dataset_dict = {'mnist':'MNIST','fmnist':'FMNIST','cifar10':''}
    define_dict = {'fedavg':'FedAVG','fedavg-krum':'Krumavg','fedsign':'sign','fedsign-krum':'Krum','fedsign-fadc-nmi':'mipc','majority-vote':'MajorityVote',
    'fedsign-mipc-mask':'mipc','majority-vote-ad':'MajorityVote_AD','dprsa-f':'DPRSA_F'}
    attack_dict = {'gaussian':'gaussian','label':'labelflip','pixel':'pixel','replace':'replace'}

    if define_type in ['dprsa-f','majority-vote'] and dataset=='cifar10':
        root_path = '/home/featurize/work/tensorboard0127/tensorboard_logs/' + dataset +'/'+ distribution +'/attack/' + attack_type 
    else:
        root_path = '/home/featurize/result/tensorboard_logs/'+ dataset +'/'+ distribution +'/attack/' + attack_type 

    if dataset == 'cifar10' and define_type == 'dprsa-f':
        r_round=500
        attack_round = 300
    if dataset == 'cifar10' and define_type == 'majority-vote':
        r_round=1000
        attack_round = 800

    if dataset == 'cifar10':
        model_n = 'alexnet'
        for i,value in define_dict.items():
            if i == 'dprsa-f':
                define_dict[i] = 'DPRSA_v2_F'
            elif i == 'fedsign-mipc-mask':
                define_dict[i] = define_dict[i]
            else:
                define_dict[i] = value + '_v2'

    # 如果有mask
    if 'mask' in define_type:
        str_mask = '_mask'
        if dataset == 'cifar10':
            str_mask = '_mask_v2'
    else:
        str_mask = ''

    if dataset == 'cifar10' or 'dprsa' in define_type:
        name = 'R_' + define_dict[define_type] + '_' + attack_dict[attack_type] + str_mask + '_' + distribution + '_'
    else:
        name = 'R_' + dataset_dict[dataset] + '_' + define_dict[define_type] + '_' + attack_dict[attack_type] + str_mask + '_' + distribution + '_'
    temp = ['attack_num:'+ str(i) + 'attack_frequency:1client_num:20train_num:20train_round:'+ str(r_round) +'(model):'+ model_n +'(experiment):'+ name + str(i) for i in range(2,20,2)]
    if attack_type == 'replace':
        temp = ['attack_round:' + str(attack_round)+ 'client_num:20train_num:20train_round:'+ str(r_round) +'(model):'+model_n+'(experiment):'+ name + str(i) for i in range(2,20,2)]
        # 'attack_round:150client_num:20train_num:20train_round:200(model):lanet(experiment):R_sign_replace_noniid_8'
    # print(root_path)

    path_list = [root_path + '/' + i for i in temp]
    # print(path_list)
    test = [len(os.listdir(i)) for i in path_list[:]]
    # print(test)

    if mean_value == True:
        event_path_list=[]
        for path in path_list:
            tmp_list = []
            tmp = path + '/'
            file_name_list = get_file_list(path)[-5:]
            for file_name in file_name_list:
                tmp_list.append(tmp+file_name)
            event_path_list.append(tmp_list)
        event_data = [get_event_data_mean(i,key) for i in event_path_list]
        pass
    else:
        event_path_list = [i + '/' + get_file_list(i)[-1]  for i in path_list[:]]
        # print(event_path_list)
        event_data = [get_event_data(i,key) for i in event_path_list]
    
    return event_data

In [47]:
[max(i[180:]) for i in get_group_data(dataset='mnist',distribution='iid',attack_type='gaussian',define_type='fedavg')]

[0.9598000049591064,
 0.9438999891281128,
 0.8992000222206116,
 0.11349999904632568,
 0.11349999904632568,
 0.11349999904632568,
 0.11349999904632568,
 0.11349999904632568,
 0.11349999904632568]

In [48]:
cov_dict = {'MNIST':'mnist','FMNIST':'fmnist','CIFAR10':'cifar10',
            'FedSIGN':'fedsign-mipc-mask','FedAVG':'fedavg',
            'Gaussian Noise Attack':'gaussian','Label Flip Attack':'label','Pixel Backdoor Attack':'pixel','Model Replacement Attack':'replace'}

all_data = pd.DataFrame()

all_data_server_acc = []
all_data_malicious = [10,20,30,40,50,60,70,80,90]*3*2*4
all_data_dataset = ['MNIST']*2*4*9+['F-MNIST']*2*4*9+['CIFAR10']*2*4*9
all_data_framework = (['FedSIGN']*4*9+['FedAVG']*4*9)*3
all_data_attack = (['Gaussian Noise Attack']*9+['Label Flip Attack']*9+['Pixel Backdoor Attack']*9+['Model Replacement Attack']*9)*2*3

for k in ['MNIST','FMNIST','CIFAR10']:
    for i in ['FedSIGN','FedAVG']:
        for j in ['Gaussian Noise Attack','Label Flip Attack','Pixel Backdoor Attack','Model Replacement Attack']:
            data = [max(i[180:]) for i in get_group_data(dataset=cov_dict[k],distribution='iid',attack_type=cov_dict[j],define_type=cov_dict[i])]
            # print(k,i,j,data)
            all_data_server_acc = all_data_server_acc + data

all_data['Main Task Accuracy'] = all_data_server_acc
all_data['Ratio of Malicious Clients (%)'] = all_data_malicious
all_data['Dataset'] = all_data_dataset
all_data['Framework'] = all_data_framework
all_data['Attack Type'] = all_data_attack

In [49]:
import plotly
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
fig = px.line(all_data, x="Ratio of Malicious Clients (%)", y="Main Task Accuracy", color='Attack Type',symbol="Attack Type", facet_row='Dataset',facet_col='Framework'
                ,markers=True,symbol_sequence=['x-open-dot','cross-open-dot','diamond-open-dot','square-open-dot'],color_discrete_sequence=colors)
fig.update_layout(height=1000,width=1000,legend=dict(orientation="h",y=1.05),)
xx = [0,10,20,30,40,50,60,70,80,90]
fig.update_xaxes(tickvals=xx)
fig.show()
fig.write_image('./r_diff_dataset_main.png', scale=4)

In [50]:
# 后门攻击成功正确率
all_data_backdoor = pd.DataFrame()

all_data_backdoor_server_acc = []
all_data_backdoor_malicious = [10,20,30,40,50,60,70,80,90]*3*2*2
all_data_backdoor_dataset = ['MNIST']*2*2*9+['F-MNIST']*2*2*9+['CIFAR10']*2*2*9
all_data_backdoor_framework = (['FedSIGN']*2*9+['FedAVG']*2*9)*3
all_data_backdoor_attack = (['Pixel Backdoor Attack']*9+['Model Replacement Attack']*9)*2*3

for k in ['MNIST','FMNIST','CIFAR10']:
    for i in ['FedSIGN','FedAVG']:
        for j in ['Pixel Backdoor Attack','Model Replacement Attack']:
            data = [max(i[190:]) for i in get_group_data(dataset=cov_dict[k],distribution='iid',attack_type=cov_dict[j],define_type=cov_dict[i],key='Attack_Acc')]
            print(k,i,j,data)
            all_data_backdoor_server_acc = all_data_backdoor_server_acc + data

all_data_backdoor['Backdoor Task Accuracy'] = all_data_backdoor_server_acc
all_data_backdoor['Ratio of Malicious Clients (%)'] = all_data_backdoor_malicious
all_data_backdoor['Dataset'] = all_data_backdoor_dataset
all_data_backdoor['Framework'] = all_data_backdoor_framework
all_data_backdoor['Attack Type'] = all_data_backdoor_attack

MNIST FedSIGN Pixel Backdoor Attack [0.11410000175237656, 0.11420000344514847, 0.11410000175237656, 0.11460000276565552, 0.1143999993801117, 0.11389999836683273, 0.1136000007390976, 0.1136000007390976, 0.11379999667406082]
MNIST FedSIGN Model Replacement Attack [0.015625, 0.171875, 0.09375, 0.125, 0.15625, 0.265625, 0.09375, 0.09375, 0.125]
MNIST FedAVG Pixel Backdoor Attack [0.9851999878883362, 0.9944000244140625, 0.9995999932289124, 0.9998999834060669, 1.0, 1.0, 1.0, 1.0, 1.0]
MNIST FedAVG Model Replacement Attack [0.828125, 0.96875, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
FMNIST FedSIGN Pixel Backdoor Attack [0.09799999743700027, 0.09830000251531601, 0.09809999912977219, 0.09799999743700027, 0.09790000319480896, 0.09809999912977219, 0.0982000008225441, 0.09780000150203705, 0.09989999979734421]
FMNIST FedSIGN Model Replacement Attack [0.015625, 0.03125, 0.09375, 0.015625, 0.015625, 0.03125, 0.03125, 0.015625, 0.0]
FMNIST FedAVG Pixel Backdoor Attack [0.9174000024795532, 0.9830999970436096

In [51]:
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
xx = [0,10,20,30,40,50,60,70,80,90]
fig = px.line(all_data_backdoor, x="Ratio of Malicious Clients (%)", y="Backdoor Task Accuracy", color='Attack Type',symbol="Attack Type", facet_row='Dataset', facet_col='Framework',
                markers=True,symbol_sequence=['diamond-open-dot','square-open-dot'],color_discrete_sequence=colors[2:])
fig.update_layout(height=1000,width=1000,legend=dict(orientation="h",y=1.05),)
fig.update_xaxes(tickvals=xx)
fig.show()
fig.write_image('./r_diff_dataset_backdoor.png', scale=4)

In [52]:
main_task = list(all_data.groupby(by=['Attack Type','Dataset','Framework']))
backdoor_task = list(all_data_backdoor.groupby(by=['Attack Type','Dataset','Framework']))

In [53]:
fig = make_subplots(rows=4,cols=3,
                    start_cell='top-left',
                    print_grid=True,
                    # x_title='111',
                    horizontal_spacing=0.1,
                    vertical_spacing=0.08,
                    subplot_titles=\
                            ("GNA (MNIST)", "LFA (MNIST)", 'PBA (MNIST)', 'MRA (MNIST)',
                            "GNA (F-MNIST)", "LFA (F-MNIST)", 'PBA (F-MNIST)', 'MRA (F-MNIST)',
                            "GNA (CIFAR10)", "LFA (CIFAR10)", 'PBA (CIFAR10)', 'MRA (CIFAR10)',)
                            )

This is the format of your plot grid:
[ (1,1) x,y     ]  [ (1,2) x2,y2   ]  [ (1,3) x3,y3   ]
[ (2,1) x4,y4   ]  [ (2,2) x5,y5   ]  [ (2,3) x6,y6   ]
[ (3,1) x7,y7   ]  [ (3,2) x8,y8   ]  [ (3,3) x9,y9   ]
[ (4,1) x10,y10 ]  [ (4,2) x11,y11 ]  [ (4,3) x12,y12 ]



In [172]:
fig = make_subplots(rows=3,cols=4,
                    start_cell='top-left',
                    # print_grid=True,
                    # x_title='111',
                    horizontal_spacing=0.07,
                    vertical_spacing=0.1,
                    column_widths=[0.25,0.25,0.25,0.25],
                    subplot_titles=\
                            ("GNA (MNIST)", "LFA (MNIST)", 'PBA (MNIST)', 'MRA (MNIST)',
                            "GNA (F-MNIST)", "LFA (F-MNIST)", 'PBA (F-MNIST)', 'MRA (F-MNIST)',
                            "GNA (CIFAR10)", "LFA (CIFAR10)", 'PBA (CIFAR10)', 'MRA (CIFAR10)',),
                    specs=[[{"secondary_y": False},{"secondary_y": False},{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": False},{"secondary_y": False},{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": False},{"secondary_y": False},{"secondary_y": True}, {"secondary_y": True}],]
                            )



colors = plotly.colors.DEFAULT_PLOTLY_COLORS
symbols = ['diamond-open-dot','square-open-dot']

cov_dist = {'MNIST':1,'F-MNIST':2,'CIFAR10':3,
            'Gaussian Noise Attack':1,'Label Flip Attack':2,'Pixel Backdoor Attack':3,'Model Replacement Attack':4}

lgroup = ['group1','group2']
lgroup_title = ['Baseline','FedSIGN']
show = True
for i, data in enumerate(main_task):

    if i>1 :
        show=False

    # print(data[0],i%2,)
    gg = i%2
    mk = dict(symbol=symbols[gg],color=colors[gg])
    trace= go.Scatter(x=data[1]['Ratio of Malicious Clients (%)'].tolist(), y=data[1]['Main Task Accuracy'].tolist(),name=data[0][2]+' - Main Task',showlegend=show,marker=mk) #,legendgroup=lgroup[gg],legendgrouptitle_text=lgroup_title[gg]
    fig.add_trace(trace,row=cov_dist[data[0][1]],col=cov_dist[data[0][0]])

#  后门的图像
show = True
for i, data in enumerate(backdoor_task):
    if i>1 :
        show=False
    gg = i%2
    # print(cov_dist[data[0][1]],cov_dist[data[0][0]])
    mk = dict(color=colors[gg])
    trace= go.Bar(x=data[1]['Ratio of Malicious Clients (%)'].tolist(), y=data[1]['Backdoor Task Accuracy'].tolist(),name=data[0][2]+' - Backdoor Task',showlegend=show,marker=mk,opacity=0.5)
    fig.add_trace(trace,row=cov_dist[data[0][1]],col=cov_dist[data[0][0]],secondary_y=True)

for i in range(1,4):
    for j in range(1,5):
        min = 0
        if i == 1:
            max = 1.02
        elif i ==2:
            max = 0.9
        else:
            max = 0.8
        
        if j>2:
            min = 0.4

        fig.update_yaxes(title_text="Main Task Accuary",tickvals=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],range=[0,1],row=i,col=j,title_standoff=3,title_font=dict(size=18),secondary_y=False)
        fig.update_yaxes(title_text="Backdoor Task Accuary",row=i,col=j,title_standoff=5,title_font=dict(size=18),secondary_y=True)
        fig.update

fig.update_xaxes(tickfont=dict(size=13),title_text='Ratio of Malicious Clients (%)', title_font=dict(size=18) ,tickvals=[0,10,20,30,40,50,60,70,80,90,100])

fig.update_layout(height=1300, width=2000,margin=dict(b=0,t=0,l=100,r=0),
                    legend=dict(orientation="h",xanchor='center',yanchor='bottom',x=0.5,y=1.04,font=dict(size=18))
    )
fig.show()
r_path = './r_diff_dataset.png'
fig.write_image(r_path, scale=4)


In [89]:
backdoor_task

[(('Model Replacement Attack', 'CIFAR10', 'FedAVG'),
       Backdoor Task Accuracy  Ratio of Malicious Clients (%)  Dataset  \
  99                 0.171875                              10  CIFAR10   
  100                0.250000                              20  CIFAR10   
  101                0.218750                              30  CIFAR10   
  102                0.343750                              40  CIFAR10   
  103                0.390625                              50  CIFAR10   
  104                0.562500                              60  CIFAR10   
  105                0.546875                              70  CIFAR10   
  106                0.671875                              80  CIFAR10   
  107                0.687500                              90  CIFAR10   
  
      Framework               Attack Type  
  99     FedAVG  Model Replacement Attack  
  100    FedAVG  Model Replacement Attack  
  101    FedAVG  Model Replacement Attack  
  102    FedAVG  Model Repla